## 라이브러리

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
import csv
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'AppleGothic'
# plt.rcParams['font.family'] = 'NanumGothic'

## vvt 파일 불러오기

In [2]:
import os
path = '../script'

# 다큐멘터리(0), 드라마(1), 영화(2) 선택
category = 2
if category == 0:
    path = path + '/Documentary'
elif category == 1:
    path = path + '/Drama'
else:
    path = path + '/Movie'

_list = os.listdir(path)
file_list = []
for file in _list:
    if file.startswith('.'): continue
    file_list.append(file)

print(len(file_list), file_list[:5])

46 ['About.Time', 'Arthur.Christmas', 'Baby.Driver', 'Baby.DriverBaby.Driver', 'Bad.Boys']


## 여기서 작품 선택합니다 꼭!!

In [3]:
# 바로 위 코드에서 file_list를 참고하여 작품 선택
title = file_list[0]

file_name = title + '.WEBRip.Netflix.en[cc]'
print(f'{path}/{title}/{file_name}')

../script/Movie/About.Time/About.Time.WEBRip.Netflix.en[cc]


## VTT 파일 읽기

In [4]:
opened_file = open(f'{path}/{title}/{file_name}.vtt', encoding='utf8')
content = opened_file.read()
segments = content.split('\n\n')    # 한줄씩 정리

In [5]:
segments[:5]

['WEBVTT',
 'NOTE Netflix\nNOTE Profile: webvtt-lssdh-ios8\nNOTE Date: 2015/07/27 20:25:13',
 'NOTE SegmentIndex\nNOTE Segment=596.458 55824@734 339\nNOTE Segment=595.334 52258@56558 323\nNOTE Segment=570.000 57352@108816 355\nNOTE Segment=588.000 53460@166168 332\nNOTE Segment=596.042 50974@219628 314\nNOTE Segment=582.083 66848@270602 411\nNOTE Segment=595.917 53618@337450 332\nNOTE Segment=597.041 55656@391068 339\nNOTE Segment=586.917 52088@446724 319\nNOTE Segment=596.875 55484@498812 343\nNOTE Segment=593.416 45924@554296 278\nNOTE Segment=596.792 38542@600220 233\nNOTE Segment=145.667 5588@638762 32\nNOTE /SegmentIndex',
 '                                                                                                 ',
 '                    \n1\n00:00:29.834 --> 00:00:32.917  position:19.52%,start  align:start size:44.29%  line:84.62% \n<c.white><c.mono_sans>[ Waves Crashing ]</c.mono_sans></c.white>']

## 불필요한 부분 제거

In [6]:
m = re.compile(r"\<.*?\>")
def clean(content):
    if '♪' in content: return None
    if '[' in content: return None
    if ']' in content: return None
    if '(' in content: return None
    if ')' in content: return None
    if '{' in content: return None
    if '}' in content: return None
    new_content = m.sub('',content)
    new_content = new_content.replace('-->','')
    remove_point_1 = new_content.find('position:')
    remove_point_2 = new_content.find('% \n')
    new_content = new_content[:remove_point_1] + new_content[remove_point_2+3:]
    return new_content

In [7]:
new_segments = [clean(s) for s in segments if len(s)!=0][4:]
new_segments[:5]

[None,
 None,
 None,
 '4\n00:00:54.291  00:00:57.208  I always knew',
 '5\n00:00:54.291  00:00:57.208  we were a fairly odd family.']

## 줄번호 / 속도 시작 / 속도 끝 / 자막 - 데이터 분리

In [8]:
trimmed_segments = []
for segment in new_segments:
    if segment != None and segment != '':
      split_segment = segment.split()
      times = split_segment[:3]
      script = split_segment[3:]
      times.append(script)
      trimmed_segments.append(times)

In [9]:
trimmed_segments[:5]

[['4', '00:00:54.291', '00:00:57.208', ['I', 'always', 'knew']],
 ['5',
  '00:00:54.291',
  '00:00:57.208',
  ['we', 'were', 'a', 'fairly', 'odd', 'family.']],
 ['6', '00:00:57.250', '00:01:02.125', ['First', 'there', 'was', 'me.']],
 ['7', '00:00:57.250', '00:01:02.125', ['Too', 'tall,', 'too', 'skinny,']],
 ['8', '00:00:57.250', '00:01:02.125', ['too', 'orange.']]]

## 데이터 저장

In [10]:
with open(f'{path}/{title}/{file_name}.csv', 'w', encoding='utf8', newline='') as f:
    for line in trimmed_segments:
        thewriter = csv.writer(f)
        thewriter.writerow(line)

## 최종 - 줄번호 / 속도 시작 / 속도 끝 / 대사 별로 컬럼 분리

In [11]:
data = pd.read_csv(f'{path}/{title}/{file_name}.csv', names=['num', 'start', 'end', 'script'])

data.head()

,num,start,end,script
0,4,00:00:54.291,00:00:57.208,"['I', 'always', 'knew']"
1,5,00:00:54.291,00:00:57.208,"['we', 'were', 'a', 'fairly', 'odd', 'family.']"
2,6,00:00:57.250,00:01:02.125,"['First', 'there', 'was', 'me.']"
3,7,00:00:57.250,00:01:02.125,"['Too', 'tall,', 'too', 'skinny,']"
4,8,00:00:57.250,00:01:02.125,"['too', 'orange.']"


In [12]:
data.to_csv(f'{path}/{title}/{file_name}.csv', columns=['start', 'end', 'script'], index=False)
data = pd.read_csv(f'{path}/{title}/{file_name}.csv')

data.head()

,start,end,script
0,00:00:54.291,00:00:57.208,"['I', 'always', 'knew']"
1,00:00:54.291,00:00:57.208,"['we', 'were', 'a', 'fairly', 'odd', 'family.']"
2,00:00:57.250,00:01:02.125,"['First', 'there', 'was', 'me.']"
3,00:00:57.250,00:01:02.125,"['Too', 'tall,', 'too', 'skinny,']"
4,00:00:57.250,00:01:02.125,"['too', 'orange.']"
